# step 2 - performed some data transformation and cleaning in pandas.

## Import the pandas library and create a DATAFRAME to work to clean up the data

In [7]:
import pandas as pd 
import datetime as dt
# loding datasets
table_match_results = pd.read_csv('datasets/table_match_results.csv')
table_batting = pd.read_csv('datasets/table_batting.csv')
table_bowling = pd.read_csv('datasets/table_bowling.csv') 
table_player = pd.read_csv('datasets/table_player.csv')


## Let's deal with the table match results

In [2]:
table_match_results.sample(n = 5)

,match,winer,margin,ground,match_date,match_ID
4,Zimbabwe VS West Indies,West Indies,inns & 4 runs,Bulawayo,"Feb 12-14, 2023",Test # 2491
20,England VS Australia,Australia,2 wickets,Birmingham,"Jun 16-20, 2023",Test # 2507
15,Sri Lanka VS Ireland,Sri Lanka,inns & 280 runs,Galle,"Apr 16-18, 2023",Test # 2502
6,India VS Australia,India,6 wickets,Delhi,"Feb 17-19, 2023",Test # 2493
1,Australia VS South Africa,drawn,-,Sydney,"Jan 4-8, 2023",Test # 2488


In [3]:

table_match_results.dtypes

match         object
winer         object
margin        object
ground        object
match_date    object
match_ID      object
dtype: object

To separate the dates and keep only the start date, we can use the `str.split()` method and then select the first element of the resulting list. Here's an example code:

```python
table_match_results['start_date'] = table_match_results['start_date'].str.split('-').str[0]

In [8]:

print(table_match_results["match_date"])
# we can see problm whut iloc[8] we need only start date
table_match_results["match_date"].iloc[8] = "Feb 28-28, 2023"

# split to "Jan 2-6" and "2030"
table_match_results[["m_d","y"]] = table_match_results["match_date"].str.split(',', expand=True)
table_match_results["y"] = table_match_results["y"].str.strip()

# split to "Jan" and "2-6"
table_match_results[["m","d"]] = table_match_results["m_d"].str.split(' ',expand = True)

# split to "2" and "6"
table_match_results[["ds","df"]] = table_match_results["d"].str.split('-',expand=True)

# conben to "02/Jan/2023"
table_match_results['start_date'] = table_match_results["ds"] + "/" + table_match_results["m"] + "/" + table_match_results["y"] 
table_match_results['start_date'] = pd.to_datetime(table_match_results['start_date'])
table_match_results.drop(columns=['match_date', 'm','y','ds','df','m_d','d'],inplace= True)

table_match_results.dtypes

0          Jan 2-6, 2023
1          Jan 4-8, 2023
2          Feb 4-8, 2023
3         Feb 9-11, 2023
4        Feb 12-14, 2023
5        Feb 16-19, 2023
6        Feb 17-19, 2023
7        Feb 24-28, 2023
8     Feb 28-Mar 2, 2023
9          Mar 1-3, 2023
10        Mar 8-11, 2023
11        Mar 9-13, 2023
12        Mar 9-13, 2023
13       Mar 17-20, 2023
14         Apr 4-7, 2023
15       Apr 16-18, 2023
16       Apr 24-28, 2023
17         Jun 1-3, 2023
18        Jun 7-11, 2023
19       Jun 14-17, 2023
20       Jun 16-20, 2023
Name: match_date, dtype: object


match                 object
winer                 object
margin                object
ground                object
match_ID              object
start_date    datetime64[ns]
dtype: object

## Let's deal with the table player

In [175]:
table_player.head()

,name,team,playingRole,battingStyle,bowlingStyle
0,Tom Latham,New Zealand,Wicketkeeper Batter,Left hand Bat,Right arm Medium
1,Devon Conway,New Zealand,Wicketkeeper Batter,Left hand Bat,Right arm Medium
2,Kane Williamson,New Zealand,Top order Batter,Right hand Bat,Right arm Offbreak
3,Henry Nicholls,New Zealand,Top order Batter,Left hand Bat,Right arm Offbreak
4,Daryl Mitchell,New Zealand,Allrounder,Moose,Right hand Bat


In [176]:
# check duplicates for name because of capabilities while scraping we scraped double values
duplicates = table_player["name"].duplicated()

# Print the duplicate rows
print(table_player[duplicates])
# Drop duplicate per "name"
table_player = table_player.drop_duplicates(subset=["name"], ignore_index= True) 
# Check that duplicates have been deleted
print(table_player[table_player["name"].duplicated()])

               name      team  ...    battingStyle                  bowlingStyle
22        Mir Hamza  Pakistan  ...   Left hand Bat               Left arm Medium
23      Naseem Shah  Pakistan  ...  Right hand Bat                Right arm Fast
24        Hasan Ali  Pakistan  ...  Right hand Bat         Right arm Medium fast
25      Abrar Ahmed  Pakistan  ...  Right hand Bat               Legbreak Googly
26      Agha Salman  Pakistan  ...  Right hand Bat            Right arm Offbreak
..              ...       ...  ...             ...                           ...
660  James Anderson   England  ...           Jimmy                 Left hand Bat
661     Harry Brook   England  ...  Right hand Bat              Right arm Medium
662       Moeen Ali   England  ...             Moe                 Left hand Bat
663      Ben Stokes   England  ...   Left hand Bat         Right arm Fast medium
664        Joe Root   England  ...  Right hand Bat  Right arm Offbreak, Legbreak

[491 rows x 5 columns]
Empt

Here we noticed that while scraping data, we scraped wrong data. That's why we made updates in this area. If the field (BowlingStyle) contained the data about a Bowling in some right-handed style, then in the field (battingStyle) record a right-handed batting style, the same for a left-handed batting

In [177]:
# Through grouping we have noticed that right handed bowling style is usually matched 
# with right batting style and the same for left handed
print(table_player.groupby("battingStyle")["bowlingStyle"].value_counts())

battingStyle     bowlingStyle                        
Akshar Patel     Left hand Bat                            1
Bluey            Right hand Bat                           1
BoBo             Right hand Bat                           1
Broady           Left hand Bat                            1
Cham             Right hand Bat                           1
Cummo            Right hand Bat                           1
Gazza            Left hand Bat                            1
Hamza Hotak      Right hand Bat                           1
Hashmat Shaidi   Left hand Bat                            1
Jimmy            Left hand Bat                            1
Josh             Right hand Bat                           1
Leachy, Nut      Left hand Bat                            1
Left hand Bat    Right arm Offbreak                      10
                 Slow Left arm Orthodox                   8
                 Right arm Medium                         4
                 Left arm Medium fast         

In [178]:
import numpy as np
batting_categories = ["Left hand Bat","Right hand Bat"] 

# We have defined a list of true/false conditions whether field (bowlingStyle) contains left/right
conditions = [(table_player["bowlingStyle"].str.contains("Left")),
              (table_player["bowlingStyle"].str.contains("Right"))
            ]
# We placed according to the condition and if the condition is not met then another.
table_player["battingStyle"] = np.select(conditions,batting_categories,default="Other")
print(table_player["battingStyle"].value_counts()) 


Right hand Bat    107
Left hand Bat      37
Other              30
Name: battingStyle, dtype: int64


## Let's deal with the table batting

In [179]:
table_batting.head()

,match,match_ID,teamInnings,battingPos,batsmanName,Out/Not Out,runs,balls,4s,6s,SR
0,Pakistan VS New Zealand,Test # 2487,Innings 1,1,Tom Latham,Out,71,100,9,0,71.00
1,Pakistan VS New Zealand,Test # 2487,Innings 1,2,Devon Conway,Out,122,191,16,1,63.87
2,Pakistan VS New Zealand,Test # 2487,Innings 1,3,Kane Williamson,Out,36,91,5,0,39.56
3,Pakistan VS New Zealand,Test # 2487,Innings 1,4,Henry Nicholls,Out,26,56,3,0,46.42
4,Pakistan VS New Zealand,Test # 2487,Innings 1,5,Daryl Mitchell,Out,3,14,0,0,21.42


In this table we are missing a group column, so we will work on that and add it

In [180]:
# Removing irrelevant characters
table_batting['batsmanName'] = table_batting['batsmanName'].apply(lambda x : x.replace("†",""))
table_batting['batsmanName'] = table_batting['batsmanName'].apply(lambda x : x.replace("(c)",""))

# Lowering spaces so that there is an adjustment for connection later
table_player["name"] = table_player["name"].str.strip()
table_batting['batsmanName'] = table_batting['batsmanName'].str.strip()

# Field isolation is relevant to connect and get the desired field
player_name_team = table_player[["name","team"]]

# Connection between two tables in order to separate the team field
tmep = table_batting.merge(player_name_team, left_on = "batsmanName", right_on = "name", how = "left")

# Redefining a table with a field team
table_batting = tmep[["match","match_ID","teamInnings","battingPos","batsmanName","Out/Not Out","runs","balls","4s","6s","SR","team"]]

## Let's deal with the table bowling

In [181]:
table_bowling.head()

,match,match_ID,teamInnings,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls
0,Pakistan VS New Zealand,Test # 2487,Innings 1,Mir Hamza,21.0,3,72,0,3.42,89,9,0,0,2
1,Pakistan VS New Zealand,Test # 2487,Innings 1,Naseem Shah,24.0,7,71,3,2.95,110,8,1,0,0
2,Pakistan VS New Zealand,Test # 2487,Innings 1,Hasan Ali,23.0,4,72,0,3.13,108,9,1,1,3
3,Pakistan VS New Zealand,Test # 2487,Innings 1,Abrar Ahmed,37.0,5,149,4,4.02,151,20,1,0,0
4,Pakistan VS New Zealand,Test # 2487,Innings 1,Agha Salman,26.0,3,75,3,2.88,109,8,0,0,0


In this table we are missing a group column, so we will work on that and add it

In [182]:
# Lowering spaces so that there is an adjustment for connection later
table_bowling["bowlerName"] = table_bowling["bowlerName"].str.strip() 

# Connection between two tables in order to separate the team field
tmep2 = table_bowling.merge(player_name_team, left_on = "bowlerName", right_on = "name", how = "left") 

# Redefining a table with a field team
table_bowling = tmep2[['match', 'match_ID', 'teamInnings','team', 'bowlerName', 'overs', 'maiden','runs', 'wickets', 'economy', '0s', '4s', '6s', 'wides', 'noBalls']] 

To create CSV files, follow these steps:

1. Import the pandas library: `import pandas as pd`
2. Create a dataframe with your desired data
3. Use the `to_csv()` method to save the dataframe as a CSV file. For example: `df.to_csv('filename.csv', index=False)`

Note: The `index=False` parameter is used to exclude the index column from the CSV file. If you want to include the index column, simply omit this parameter or set it to `True`.

In [183]:
# Creating CSV files Go to step 2 of the project    
file_name1 = 'datasets/table_match_results_clean.csv'
file_name2 = 'datasets/table_batting_clean.csv' 
file_name3 = 'datasets/table_bowling_clean.csv'
file_name4 = 'datasets/table_player_clean.csv' 

table_match_results.to_csv(file_name1, index=False)
table_batting.to_csv(file_name2, index=False)
table_bowling.to_csv(file_name3, index=False)
table_player.to_csv(file_name4, index=False)